### 라이브러리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain-community
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install tiktoken
!pip install langchain-google-genai
!pip install chromadb
!pip install langchain_huggingface
!pip install ragas
!pip install faiss-cpu
!pip install faiss-gpu
# !pip install pymilvus  # Milvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.6 MB/s eta 0:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import os
import re
import pandas as pd
import bs4
import tiktoken
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output
import time

# 기존 FAISS 대신 Milvus 사용
# from langchain.vectorstores import Milvus
# from pymilvus import connections, utility

from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate

### API Key

In [ ]:
# gemini
YOUR_API_KEY = ''
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

### 데이터

In [ ]:
# CSV
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250311_semifinal.csv"
df = pd.read_csv(file_path)

In [ ]:
df.columns

Index(['Unnamed: 0', 'ISBN', '분류', '제목', '부제', '저자', '발행자', '발행일', '페이지', '가격',
       '표지', '책소개', '저자소개', '목차', '출판사리뷰', '추천사'],
      dtype='object')

In [ ]:
df = df.drop(columns=['Unnamed: 0','발행자','발행일','표지'])

In [ ]:
# df = df.sample(n=20000, random_state=2025)

### Chunking

In [ ]:
# 텍스트 분할 함수 (null 값은 빈 문자열로 처리)
def split_text(text, chunk_size=1000, overlap=100):
    if text is None or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [ ]:
# 메타데이터에 포함될 컬럼과 벡터 DB Documents에 들어갈 핵심 데이터 컬럼 정의

# metadata_columns = ['ISBN', 'ITEM_ID', 'BID', 'GOODS_NO', '발행자', '발행일', '페이지', '가격', '표지', 'INSERT_DATE', 'UPDATE_DATE'] # 원제
# vector_doc_columns = ['제목', '분류', '저자', '책 소개', '저자소개', '목차', '출판사리뷰']

metadata_columns = ['ISBN', '페이지', '가격', '제목', '저자']
vector_doc_columns = ['제목', '분류', '저자','저자소개', '책소개', '목차', '출판사리뷰','추천사']

In [ ]:
# RAG_DB 구성: 각 행의 vector_doc_columns를 하나의 텍스트로 합치고, metadata_columns에 해당하는 데이터는 별도 dict에 저장

# text는 아래와 같은 형태로 들어가서 청크

# 제목 : [값]
# 분류 : [값]
# 저자 : [값]
# 책 소개 : [값]

# 목차 : [값]
# 출판사리뷰 : [값]


RAG_DB = []
for index, row in df.iterrows():
    # 핵심 데이터(문서 내용) 생성: 각 컬럼명과 값을 줄바꿈 형태로 연결
    doc_text = ""
    for col in vector_doc_columns:
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        doc_text += f"{col} : {value}\n"

    # 텍스트 분할
    chunks = split_text(doc_text)

    # 메타데이터 구성: metadata_columns에 있는 모든 데이터를 개별적으로 저장
    metadata = {}
    for col in metadata_columns:
        metadata[col] = row.get(col, None)

    # 분할된 각 청크를 RAG_DB에 추가
    for chunk in chunks:
        RAG_DB.append({
            'text': chunk,
            'metadata': metadata
        })

In [ ]:
# LangChain Document 생성: 각 RAG_DB 항목의 text와 metadata를 그대로 사용
from langchain.docstore.document import Document

documents = [
    Document(
        page_content=entry['text'],
        metadata=entry['metadata']
    ) for entry in RAG_DB
]

### 임베딩 및 벡터스토어 생성

In [ ]:
# 임베딩 모델 로드 (HuggingFace의 BGE-m3)
hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 벡터스토어 생성을 위해 텍스트와 임베딩 쌍 구성
text_embedding_pairs = [] # (텍스트, 임베딩) 튜플 리스트
metadata_list = []

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# # Milvus 벡터스토어 생성 (FAISS 대신 사용)

# # https://person-tutorials-just-testament.trycloudflare.com

# # collection_name은 원하는 이름으로 지정하고, 필요 시 connection_args에 Milvus 서버 정보를 입력하세요.
# vectorstore = Milvus.from_documents(documents, hf_embeddings, collection_name="aiffel_books")

# # Gemini-1.5-flash
# llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

### Milver Retrival

In [ ]:
# # Milvus 기반 retriever 생성 top k : 5
# dense_retriever = vectorstore.as_retriever(
#     search_kwargs={"k": 5}
# )

# # RetrievalQA 체인 구성 - 검색 문서도 같이 반환
# dpr_qa_chain = RetrievalQA.from_chain_type(
#     llm=llm_gemini,
#     retriever=dense_retriever,
#     return_source_documents=True
# )

### FAISS

In [ ]:
for doc in tqdm(documents, desc="Processing Documents", unit="document",leave=False):
    embedding = hf_embeddings.embed_query(doc.page_content)  # 텍스트 청크 임베딩(문서의 내용)
    text_embedding_pairs.append((doc.page_content, embedding)) # 텍스트 원문과 임베딩 묶어놓기 (튜플)
    metadata_list.append(doc.metadata) # 메타데이터 임베딩 X

KeyboardInterrupt: 

In [ ]:
import os
import pickle

# 배치 사이즈
BATCH_SIZE = 17000

def process_batch(batch_number, documents, hf_embeddings, embedding_dir):
    """
    주어진 배치 번호에 해당하는 문서들을 임베딩하여 저장

    Parameters:
    - batch_number (int): 처리할 배치 번호 (1부터 시작)
    - documents (list): 문서 리스트
    - hf_embeddings: 임베딩 모델 (HuggingFaceEmbeddings 인스턴스)
    - embedding_dir (str): 저장할 디렉토리 경로
    """
    # 시작 인덱스와 끝 인덱스 계산
    start_idx = (batch_number - 1) * BATCH_SIZE
    end_idx = min(batch_number * BATCH_SIZE, len(documents))

    if start_idx >= len(documents):
        print("더 이상 처리할 문서가 없음")
        return

    batch_docs = documents[start_idx:end_idx]
    batch_text_embedding_pairs = []
    batch_metadata_list = []

    print(f"Processing batch {batch_number}: documents {start_idx} ~ {end_idx - 1} (총 {len(batch_docs)}개)")
    for doc in tqdm(batch_docs, desc="Processing Documents in batch", unit="document", leave=False):
        embedding = hf_embeddings.embed_query(doc.page_content)
        batch_text_embedding_pairs.append((doc.page_content, embedding))
        batch_metadata_list.append(doc.metadata)


    file_path = os.path.join(embedding_dir, f"embedding_results_batch_{batch_number}.pkl")
    embedding_data = {
        "text_embedding_pairs": batch_text_embedding_pairs,
        "metadata_list": batch_metadata_list
    }
    with open(file_path, "wb") as f:
        pickle.dump(embedding_data, f)
    print(f"Saved batch {batch_number} results to {file_path}")

# 사용 경로
embedding_dir = "/content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/embedding"
os.makedirs(embedding_dir, exist_ok=True)

In [ ]:
process_batch(7, documents, hf_embeddings, embedding_dir)

Processing batch 7: documents 102000 ~ 118999 (총 17000개)


Saved batch 7 results to /content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/embedding/embedding_results_batch_7.pkl


In [ ]:
# 임베딩 결과가 저장된 디렉토리
embedding_dir = "/content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/embedding"

all_text_embedding_pairs = []
all_metadata_list = []

# 저장된 모든 .pkl 파일을 불러와서 병합
for file_name in os.listdir(embedding_dir):
    if file_name.endswith(".pkl"):
        file_path = os.path.join(embedding_dir, file_name)
        with open(file_path, "rb") as f:
            data = pickle.load(f)
            all_text_embedding_pairs.extend(data["text_embedding_pairs"])
            all_metadata_list.extend(data["metadata_list"])

print(f"불러온 임베딩 문서 수: {len(all_text_embedding_pairs)}")

# FAISS 벡터스토어 생성 (미리 계산된 임베딩을 사용하므로 빠르게 생성됨)
vectorstore = FAISS.from_embeddings(
    text_embeddings=all_text_embedding_pairs,
    metadatas=all_metadata_list,
    embedding=hf_embeddings
)

print("FAISS 벡터스토어 생성 완료.")

In [ ]:
# FAISS 벡터스토어 생성
# vectorstore = FAISS.from_embeddings(
#     text_embeddings=text_embedding_pairs,
#     metadatas=metadata_list,
#     embedding=hf_embeddings
# )
vectorstore = FAISS.from_embeddings(
    text_embeddings=all_text_embedding_pairs,
    metadatas=all_metadata_list,
    embedding=hf_embeddings
)

In [ ]:
# 저장할 경로 설정
save_path = "/content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/faiss_index"

# FAISS 벡터스토어를 지정된 경로에 저장
vectorstore.save_local(save_path)

print(f"FAISS 벡터스토어가 {save_path} 에 저장")

FAISS 벡터스토어가 /content/drive/MyDrive/aiffel_final_project/data_renew/faiss_index 에 저장


In [ ]:
# 저장된 경로 설정
save_path = "/content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/faiss_index"

# 저장된 FAISS 벡터스토어 불러오기
vectorstore = FAISS.load_local(save_path, hf_embeddings, allow_dangerous_deserialization=True)

print("FAISS 벡터스토어 불러오기")

FAISS 벡터스토어 불러오기


### Retrieval

In [ ]:
# Gemini-1.5-flash
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# FAISS 벡터스토어 retriever 생성 top k : 5
dense_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

# RetrievalQA 체인 구성 - 검색 문서도 같이 반환
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_gemini,
    retriever=dense_retriever,
    return_source_documents=True
)

### Generation

In [ ]:
# 대화 히스토리와 사용자 선호도
user_preferences = defaultdict(list)
log_history = []  # 전체 대화 로그 옵션

# 멀티턴 프롬프트 템플릿 (llm이 분기별 작동하게 하는 입력 변수 명시)
multi_turn_prompt = PromptTemplate(
    input_variables=["history", "query", "impormation", "strategy", "if", "else"],
    template="""
사용자와의 대화 히스토리 :
{history}

사용자의 마지막 질문 :
"{query}"

role :
{impormation}
아래 정보를 바탕으로, 너는 사용자 선호도를 명확히 파악하고, 단호하게 검색 조건을 평가하라.
반드시 아래 기준에 따라 행동해야 한다.

1. 사용자가 책을 찾는 이유를 파악하라.
   - (1) 단순 독서를 원함.
   - (2) 정보나 기술 습득을 위해 찾음.
   - (3) 흥미 또는 취미 생활의 일환으로 찾음.

2. 사용자가 찾고자 하는 책에 대한 사전 지식 수준을 평가하라.
   - (1) 전혀 모름.
   - (2) 카테고리, 작가, 관련 책 이름 등 일부 정보를 보유.
   - (3) 구체적으로 찾고 싶은 책이 있음.
   해당 정보를 간결하게 요약하여 검색 쿼리에 활용해라.

3. 사용자의 취향에 적합한 카테고리를 구체적으로 파악하라.
   아래 항목 중 해당하는 것을 반드시 언급하라.
   - 소설: 현대, 고전, 판타지, SF, 미스터리, 로맨스, 사회 비판, 심리/철학, 역사, 전쟁 소설 등
   - 경제/경영: 기업 경영, 주식, 마케팅, 경제학 입문, 창업, 노동 철학 등
   - 자기계발: 시간관리, 멘탈 관리, 습관 형성, 리더십, 자기 탐색 등
   - 시/에세이: 철학적, 문학적, 감성, 자전적, 여행 시집 등
   - 인문/교양: 철학, 역사, 사회, 심리, 종교/명상 등
   - 취미/실용: 요리, 운동, DIY, 사진, 글쓰기, 음악/예술 등
   - 어린이/청소년: 그림책, 초등 필독서, 청소년 소설, 과학/탐구, 경제/교양 등
   반드시 구체적인 키워드를 포함하라.

{if}
사용자 쿼리를 받았을 때 검색을 할 수 있는 기준

{strategy}
아래 기준에 따라 최종 검색 쿼리를 생성하라:
1. 대화 내용을 종합하여 검색 수행 확률을 0에서 1 사이의 값으로 평가하라. 점수의 증가는 세밀하게 진행해라. (사용자가 원하는 카테고리를 안다면, 0.15점 증가!! 사용자가 책을 원하는 이유를 안다면, 0.2점 증가!! 등)확률이 0.8 이상이면 반드시 검색을 진행하라.
2. 사용자와 멀티챗(사용자 query & LLM 답변)을 10~40 단어로 요약해서 최종 쿼리(query)를 생성해라. 2~5개의 핵심 키워드는 쿼리에 반드시 포함시켜라.
3. 요약문에는 반드시 "책 추천 기준"을 반영하라. 예를 들어, "SF 장르 중 AI 관련 최신 베스트셀러" 또는 "역사 사실에 충실한 근현대사 도서"와 같이 구체적으로 기술하라.
4. 최종 검색 쿼리를 생성할 때, 반드시 추출된 핵심 키워드를 포함하고, 사용자 목적에 맞게 최종 쿼리를 확정하라.
    - 기술 서적을 원한다면, 키워드를 적절하게 나열해라.
    - 소설/자기개발서 등을 원한다면 적절하게 문장 형태로 쿼리를 조합해라.
{else}
현재까지 멀티챗을 종합했을 때, 사용자 선호도를 알 수 없다면, 5번 문항을 명심해라.

5. 만약 사용자 정보가 충분하지 않다면, {impormation}과 {strategy}에 기반하여, 검색 점수를 높일 수 있는 방향으로 적절한 보충 질문을 하나 생성하라.
    - 예시: "관심 있는 도서 카테고리를 명확히 알려줘." 또는 "찾고 있는 책의 목적을 구체적으로 말해줘." 또는 "재미있게 읽었던 책은 뭐야?"
    - 예시: "(카테고리를 모른다.) : 네가 관심있어하는 도서 카테고리가 뭐야?"
    - 예시: "(사용자의 정확한 목적을 모른다.) : 네가 찾고 있는 지식 도서는 초급자 용이야? 상급자 용이야?"

6. 추천 이유 {{reason}}을 작성할때는 책소개,저자,저자소개,출판사리뷰,추천사 등을 적절하게 활용하도록해.
    - 추천 이유에 적합한 내용을 보고 generation 해라.

[출력 형식]
최종 출력은 반드시 아래 형식을 준수하라.
1. 검색 확률: {{score}}
2. 검색 쿼리: "{{final_search_query}}"
3. 추가 질문: "{{follow_up_question}}"
4. 추천 이유: "{{reason}}"
"""
)
search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)

<ipython-input-20-b6d29ab224e6>:69: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)


In [ ]:
# 최종 검색 query 프롬프트 (fallback)
final_query_prompt = PromptTemplate(
    input_variables=["history", "fallback"],
    template="""
지금까지의 대화 내용을 바탕으로, 사용자의 선호도와 요청을 반영하여 검색에 적절한 최종 쿼리를 생성해.
1. 대화 내용을 요약하고 핵심 키워드를 추출해.
2. 추출된 핵심 키워드를 포함하여, 사용자 목적에 맞는 구체적인 검색 쿼리를 생성해.
대화 내용:
{history}
기본 검색 쿼리: {fallback}
최종 검색 쿼리:
"""
)
final_query_chain = LLMChain(llm=llm_gemini, prompt=final_query_prompt)

In [ ]:
def extract_field(text, field_name):
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

MIN_INFO_LENGTH = 10  # 최소 정보 길이 기준 - 너무 길게하면 작동 X

# def generate_answer(query):
#     result = dpr_qa_chain.invoke(query)
#     formatted_answers = []

#     for doc in result['source_documents']:
#         metadata = doc.metadata

#         title = metadata.get("제목") or extract_field(doc.page_content, "제목")
#         author = metadata.get("저자") or extract_field(doc.page_content, "저자")
#         publisher_review = extract_field(doc.page_content, "출판사리뷰")
#         book_intro = extract_field(doc.page_content, "책소개")

#         if publisher_review and book_intro:
#             combined_info = publisher_review + "\n" + book_intro
#         elif publisher_review:
#             combined_info = publisher_review
#         elif book_intro:
#             combined_info = book_intro
#         else:
#             combined_info = ""

#         if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
#             reason = "추천 이유 정보 없음"
#         else:
#             reason_prompt = (
#                 f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해줘. "
#                 f"책의 특징이나 강점을 중심으로 설명해주면 좋겠어. 만약 제공된 정보가 충분하지 않다면, '추천 이유 정보 없음'이라고 응답해줘.\n\n정보:\n{combined_info}"
#             )
#             reason_response = llm_gemini.invoke(reason_prompt)
#             generated_reason = reason_response.text().strip()

#             if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
#                 reason = "추천 이유 정보 없음"
#             else:
#                 reason = generated_reason

#         formatted = f"{title}\n{author}\n추천 이유: {reason}"
#         formatted_answers.append(formatted)

#     answer = "\n\n".join(formatted_answers)
#     return answer, None

In [ ]:
## 동일 ISBN값이면 동일한 생성으로 포함되도록

def generate_answer(query):
    # 기존 retrieval을 수행하여 상위 청크들 가져오기
    result = dpr_qa_chain.invoke(query)
    source_docs = result['source_documents']

    # retrieval된 청크들의 ISBN을 집합으로 추출 (중복 제거)
    retrieved_isbns = set()
    for doc in source_docs:
        isbn = doc.metadata.get("ISBN")
        if isbn:
            retrieved_isbns.add(isbn)

    # 각 ISBN별로 전체 documents에서 해당하는 모든 청크를 집계
    aggregated_docs = []
    for isbn in retrieved_isbns:
        # 전체 문서 리스트에서 같은 ISBN을 가진 문서들 필터링
        book_docs = [doc for doc in documents if doc.metadata.get("ISBN") == isbn]
        if not book_docs:
            continue
        # 해당 책의 모든 청크 텍스트를 하나로 합침
        aggregated_text = "\n".join([doc.page_content for doc in book_docs])
        # 메타데이터는 동일 책이라면 첫 번째 문서의 메타데이터 사용
        aggregated_docs.append(Document(page_content=aggregated_text, metadata=book_docs[0].metadata))

    # 각 책별로 추천 정보를 생성
    formatted_answers = []
    for doc in aggregated_docs:
        metadata = doc.metadata

        title = metadata.get("제목") or extract_field(doc.page_content, "제목")
        author = metadata.get("저자") or extract_field(doc.page_content, "저자")
        publisher_review = extract_field(doc.page_content, "출판사리뷰")
        book_intro = extract_field(doc.page_content, "책소개")

        # 책 소개와 출판사 리뷰를 결합하여 추천 이유 생성에 활용
        if publisher_review and book_intro:
            combined_info = publisher_review + "\n" + book_intro
        elif publisher_review:
            combined_info = publisher_review
        elif book_intro:
            combined_info = book_intro
        else:
            combined_info = ""

        if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
            reason = "추천 이유 정보 없음"
        else:
            reason_prompt = (
                f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해줘. "
                f"책의 특징이나 강점을 중심으로 설명해주면 좋겠어. 만약 제공된 정보가 충분하지 않다면, '추천 이유 정보 없음'이라고 응답해줘.\n\n정보:\n{combined_info}"
            )
            reason_response = llm_gemini.invoke(reason_prompt)
            generated_reason = reason_response.text().strip()
            if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
                reason = "추천 이유 정보 없음"
            else:
                reason = generated_reason

        formatted = f"{title}\n{author}\n추천 이유: {reason}"
        formatted_answers.append(formatted)

    answer = "\n\n".join(formatted_answers)
    return answer, None

In [ ]:
# 사용자 선호도 카테고리화 함수 - 옵션 추가
def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)

In [ ]:
def robust_parse_llm_response(response_text):
    cleaned_text = re.sub(r'\*\*', '', response_text)
    search_score = None
    search_query = None
    follow_up_question = ""

    score_match = re.search(r"검색\s*확률[:：]\s*([\d\.]+)", cleaned_text)
    if score_match:
        try:
            search_score = float(score_match.group(1))
        except Exception as e:
            print("검색 확률 파싱 에러:", e)

    query_match = re.search(r"검색\s*쿼리[:：]\s*(.*)", cleaned_text)
    if query_match:
        search_query = query_match.group(1).strip()
        if search_query.startswith('"') and search_query.endswith('"'):
            search_query = search_query[1:-1].strip()

    follow_match = re.search(r"추가\s*질문[:：]\s*(.*)", cleaned_text)
    if follow_match:
        follow_up_question = follow_match.group(1).strip()
        if follow_up_question in ["(필요 없음)", "(없음)", ""]:
            follow_up_question = ""
        if follow_up_question.startswith('"') and follow_up_question.endswith('"'):
            follow_up_question = follow_up_question[1:-1].strip()

    return search_score, search_query, follow_up_question

In [ ]:
def search_and_generate_answer(query, query_history):
    while True:
        query_summary = "\n".join(query_history[-5:])  # 최근 5개 대화 요약 - 대화가 너무 길어지지 않도록
        search_decision_dict = search_query_chain.invoke({
            "history": query_summary,
            "query": query,
            "if": "✅ 검색이 가능한 경우:",
            "else": "❌ 아직 검색이 불가능한 경우:",
            "impormation": "📌 사용자 선호도 분석:",
            "strategy": "🔍 검색 전략:",
        })
        response_text = search_decision_dict["text"].strip()
        print("\n[🔍 LLM 응답 확인]\n", response_text)

        search_score, base_search_query, follow_up_question = robust_parse_llm_response(response_text)
        print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")

        if search_score is None:
            print("\n[❌ LLM 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

        if search_score >= 0.7 and base_search_query:
            final_search_query = final_query_chain.invoke({
                "history": "\n".join(query_history),
                "fallback": base_search_query
            })["text"].strip()
            print(f"\n[🔎 최종 검색 쿼리 생성]: {final_search_query}")

            answer, sources = generate_answer(final_search_query)
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{answer}\n\n[📚 책 정보]\n{book_info}"
                print("\n[📚 책 정보]\n", book_info)
            else:
                answer_with_info = answer
            return answer_with_info

        if follow_up_question:
            print(f"\n[🤖 보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[📚 사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue

        if search_score < 0.7 or not base_search_query:
            print("\n[❌ 검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [ ]:
def interactive_multi_turn_qa():
    query_history = []  # 초기화

    while True:
        clear_output(wait=True)
        print("📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)

        query = input("질문을 입력하세요: ")

        if query.lower() == 'quit':
            print("\n[📝 대화 저장 중...]")
            log_history.append(query_history)
            print("대화 저장 완료")
            break

        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)

        print("\n[💬 AI의 답변]")
        print(answer)

        query_history.append(f"AI: {answer}")

        input("\n-> 계속하려면 Enter를 누르세요...")

In [ ]:
def show_log_history():
    print("\n[ 전체 대화 로그]")
    for i, session in enumerate(log_history, 1):
        print(f"\n 대화 세션 {i}:\n")
        print("\n".join(session))
        print("-" * 50)

In [ ]:
# 실행
interactive_multi_turn_qa()

📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)
--------------------------------------------------
질문을 입력하세요: 역사책을 찾고 있어.

[🔍 LLM 응답 확인]
 1. 검색 확률: 0.35

2. 검색 쿼리: "역사책 추천,  역사 사실에 충실한 서술 방식의 책,  초중급 수준의 세계사 또는 한국사 책"

3. 추가 질문: "어떤 역사 분야에 관심 있으세요? (예: 세계사, 한국사, 특정 시대, 특정 인물 등)  그리고 역사책을 읽는 목적이 무엇인가요? (예: 단순히 역사를 배우고 싶다, 특정 사건에 대해 자세히 알고 싶다,  흥미로운 이야기를 읽고 싶다 등)"

4. 추천 이유: 사용자는 단순히 "역사책을 찾고 있다"고만 했으므로,  책을 찾는 이유(1,2,3 중 어느 것인지),  지식 수준(1,2,3 중 어느 것인지),  구체적인 역사 분야(세계사, 한국사, 고대사 등)에 대한 정보가 부족합니다. 따라서 검색 확률은 낮습니다.  추가 질문을 통해 사용자의 선호도를 명확히 파악하여 검색 쿼리를 더욱 구체화할 필요가 있습니다. 현재 쿼리는  일반적인 역사책을 추천하는 수준으로,  사실에 충실한 서술 방식과 초중급 수준이라는 제한적인 조건을 추가하여 검색 결과의 질을 높이고자 했습니다.

[디버그] 파싱 결과: 검색 확률=0.35, 기본 검색 쿼리='역사책 추천,  역사 사실에 충실한 서술 방식의 책,  초중급 수준의 세계사 또는 한국사 책', 추가 질문='어떤 역사 분야에 관심 있으세요? (예: 세계사, 한국사, 특정 시대, 특정 인물 등)  그리고 역사책을 읽는 목적이 무엇인가요? (예: 단순히 역사를 배우고 싶다, 특정 사건에 대해 자세히 알고 싶다,  흥미로운 이야기를 읽고 싶다 등)'

[🤖 보충 질문: 어떤 역사 분야에 관심 있으세요? (예: 세계사, 한국사, 특정 시대, 특정 인물 등)  그리고 역사책을 읽는 목적이 무엇인가요? (예: 단순히 역사를 배우고 싶다, 특정 사건에 대해 

KeyboardInterrupt: Interrupted by user

In [ ]:
# 역사책을 찾고 있어.
# 한국사. 특히 조선시대면 좋겠어. 그냥 한국인으로 역사를 알아야겠다고 생각해서 찾고있어.
# -> 책소개/추천사리뷰 등이 없다면, retrival에 있어서 유사도 청크로 상위에 뽑힐 수 없을 것!!!!!!